In [18]:
import json
import os
import glob
import pandas as pd
import time
import xmltodict
import forecastio
import datetime
from tqdm import tqdm
pd.set_option('display.max_columns', 50)

In [19]:
os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/')

succeed_ls = glob.glob('*')
            

succeed_ls2 = []
failed_ls2 = []


for tourny in succeed_ls:
    #try:
    
    tourny_path = os.chdir('/home/valesco/Datasets/PGA_Data/PGA_Dir/' + tourny)
    tournament_json = open(tourny + '_course_data1.json')
    tournament_dict = json.load(tournament_json)
    
    tourny_number = tourny.split('_')[0]
    tourny_name = tournament_dict[tourny_number]['tournament_name']
    
    year = tourny.split('_')[1]

    courses = int(tournament_dict[tourny_number]['course_info']['_num_courses'])

    player_list = tournament_dict[tourny_number]['field'].keys()

    tee_time_df = pd.DataFrame(columns = ['player_id', 'player_name', 'tournament_name', 
                                          'tournament_id', 'course_name', 'course_num', 'time_zone', 'round_id', 'year',
                                          'round','date', 'tee_time', 'tee_segment', 'num_players_group',
                                          'vs_player1', 'vs_player2', 'vs_player3',
                                          'weather_visib', 'weather_precip_possib', 'weather_icon', 
                                          'weather_wind_speed', 'weather_dew_point', 'weather_temp', 
                                          'weather_pressure', 'weather_apparent_temp', 'weather_summary', 
                                          'weather_precip_inten', 'weather_cloud_cover', 'weather_wind_bearing',
                                          'weather_humidity', 'weather_time_str'])
    count = 0

    for player in player_list:
        for r, rnd in enumerate(tournament_dict[tourny_number]['tee_data']['rounds']):
            for course in rnd['courses']:
                course_name = course['CourseName']
                course_num = course['CourseNumber']
                time_zone = tournament_dict[tourny_number]['course_info']['course'][0]['course_loc']['timezone']
                if len(course_num) == 1:
                    course_num = '00' + course_num
                if len(course_num) == 2:
                    course_num = '0' + course_num

                for segment in course['segments']:
                    tee_segment = segment['SegmentID']

                    for group in segment['groups']:
                        player_id_ls = []
                        for player2 in group['players']:
                            if player == player2['TournamentPlayerID']:
                                tee_time = group['StartTime']
                                tee_date = group['StartDate']
                                for player3 in group['players']:
                                    if player != player3['TournamentPlayerID']:
                                        player_id_ls.append(player3['TournamentPlayerID'])

                                num_players_in_group = len(player_id_ls)

                                player_vars = ['vs_player1', 'vs_player2', 'vs_player3']

                                for v, var in enumerate(player_vars):
                                    try:
                                        globals()[var] = player_id_ls[v]
                                    except:
                                        globals()[var] = 'null'


                                name = player2['LFName']
                                round_id = '{}_{}_{}_{}_{}'.format(player, year, tourny_number, course_num, r + 1)


                                #Create time string for weather
                                
                                temp_time = tee_time.split(':')
                                hour = int(temp_time[0])
                                minute = int(temp_time[1].split(' ')[0])
                                

                                if int(minute) > 29:
                                    hour = hour + 1
                                else:
                                    hour = hour

                                if 'pm' in tee_time and hour < 12:
                                    hour = hour + 12

                                minute = '00'
                                hour = str(hour)


                                if len(hour) < 2:
                                    hour = '0' + hour

                                weather_time = '{}:{}'.format(hour, minute)
                                weather_str = tee_date + ' ' + weather_time

                                for tourny_dict in tournament_dict[tourny_number]['weather'].items():
                                    for t_dict in tourny_dict[1]:
                                        if t_dict['time_str'] == weather_str:
                                            try:
                                                visib = t_dict['visibility']
                                            except:
                                                visib = 'null'
                                            try:
                                                precip = t_dict['precipProbability']
                                            except:
                                                precip = 'null'
                                            try:
                                                icon = t_dict['icon']
                                            except:
                                                icon = 'null'
                                            try:
                                                wind_speed = t_dict['windSpeed']
                                            except:
                                                wind_speed = 'null'
                                            try:
                                                dew_point = t_dict['dewPoint']
                                            except:
                                                dew_point = 'null'
                                            try:
                                                temp = t_dict['temperature']
                                            except:
                                                temp = 'null'
                                            try:
                                                pressure = t_dict['pressure']
                                            except:
                                                pressure = 'null'
                                            try:
                                                app_temp = t_dict['apparentTemperature']
                                            except:
                                                app_temp = 'null'
                                            try:
                                                summary = t_dict['summary']
                                            except:
                                                summary = 'null'
                                            try:
                                                precip_int = t_dict['precipIntensity']
                                            except:
                                                precip_int = 'null'
                                            try:
                                                cloud = t_dict['cloudCover']
                                            except:
                                                cloud = 'null'
                                            try:
                                                wind_bearing = t_dict['windBearing']
                                            except:
                                                wind_bearing = 'null'
                                            try:
                                                humidity = t_dict['humidity']
                                            except:
                                                humidity = 'null'



                                tee_time_df.loc[count] = [player, name, tourny_name,
                                            tourny_number, course_name, course_num, time_zone,
                                            round_id, year, r + 1, tee_date, tee_time, tee_segment,
                                            num_players_in_group, vs_player1, vs_player2, vs_player3, visib, precip,
                                            icon, wind_speed, dew_point, temp, pressure, app_temp, 
                                            summary, precip_int, cloud, wind_bearing, humidity, weather_str]

                                count += 1

    file_name = tourny + '_player_round_data.csv'
    tee_time_df.to_csv(file_name)

    print(tourny, ' succeeded!!')
    succeed_ls2.append(tourny)
    #except Exception as e:
        #print(tourny, ' failed, exception: ', e)
        #failed_ls2.append(tourny)

020_2016  succeeded!!
005_2016  succeeded!!
010_2016  succeeded!!
041_2014  succeeded!!
003_2014  succeeded!!
023_2014  succeeded!!
025_2016  succeeded!!
473_2017  succeeded!!
457_2015  succeeded!!
010_2014  succeeded!!
018_2014  succeeded!!
002_2017  succeeded!!
033_2016  succeeded!!
473_2014  succeeded!!
020_2017  succeeded!!
060_2015  succeeded!!
473_2015  succeeded!!
054_2015  succeeded!!
025_2014  succeeded!!
007_2016  succeeded!!
011_2014  succeeded!!
025_2015  succeeded!!
483_2014  succeeded!!
010_2017  succeeded!!
034_2014  succeeded!!
013_2016  succeeded!!
006_2015  succeeded!!
480_2014  succeeded!!
018_2017  succeeded!!
002_2015  succeeded!!
475_2014  succeeded!!
011_2015  succeeded!!
032_2015  succeeded!!
013_2014  succeeded!!
505_2014  succeeded!!
030_2015  succeeded!!
016_2016  succeeded!!
480_2017  succeeded!!
030_2014  succeeded!!
518_2015  succeeded!!
472_2014  succeeded!!
475_2016  succeeded!!
032_2014  succeeded!!
019_2014  succeeded!!
020_2014  succeeded!!
030_2016  

In [15]:
tournament_dict[tourny_number]['tournament_name']

'OHL Classic at Mayakoba'

In [12]:
tourny_dict['visibility']

TypeError: tuple indices must be integers or slices, not str

In [21]:
datetime.datetime.now().year

2017